# Alexandra Gavrilina

# HOMEWORK 2: COALESCENT WITH MUTATION

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random
import math
import statistics
import scipy
from scipy import special

## 1. Basics

Chromosome is an interval $[0, 1]$.

Individual (or individual’s genome) is a set of $M$ chromosomes, numbered from $0$ to $M − 1$.

Chromosomes with the same id (from different individuals) are related by a single tree genealogy (no recombination).

Genealogies for chromosomes with different ids are simulated independently from each other.

## 2. Coalescence with mutation
Let there be $K$ lineages. Mutation rate is $\mu$, effective population size over time is $ν(t)$. Assume that $ν(t)$ is piecewise constant function.

Coalescence with mutation is a Poisson process with the (variable) rate

$$\omega(K, t) = K \mu + \dfrac{1}{v(t)} \begin{pmatrix} K \\ 2 \end{pmatrix}.$$

Simulation scheme.

(1) Set $t = 0$, initialise $K$.

(2) Sample time $T$ till the next event from Poisson process with the rate $\omega (K, t)$. Set $t = t + T$.

(3) Generate type of the event following Bernoulli distribution with weights proportional to Kµ (mutation) and $\dfrac{1}{v(t)} \begin{pmatrix} K \\ 2 \end{pmatrix}$ (coalescence).

* Mutation: sample ancestral lineages $l$ where mutation occurs independently from $K$ available lineages. Sample mutation position $p$ on a genome uniformly on $[0, 1]$. All individuals which are decedents of $l$ get variant $1$ at position $p$. All other individuals have variant $0$ at position $p$.

* Coalescence. Choose uniformly a random pair of lineages $l_1$ and $l_2$. These two lineages coalesce at time $t$. Update genealogy. Set $K = K − 1$.

(4) stop if $K = 1$. Otherwise go to step 2.

In [2]:
M = 1 # для начала
K = 2 # для начала
mu = 0 # для первой версии - без мутации

In [3]:
def v(t): # piecewise constant function
    return 0*t + 1

In [4]:
'''
Rate
Input: ...
Output: w(K, mu)
'''
def rate(K, mu, v):
    return K*mu + (1/v)*scipy.special.binom(K, 2)

In [5]:
def v_t(t):
    return np.piecewise(t, (t < 100, t >= 100), (1, 2))

https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.binom.html#scipy.special.binom

Coalescence. Choose uniformly a random pair of lineages $l_1$ and $l_2$. These two lineages coalesce at time $t$. Update genealogy. Set $K = K − 1$.

In [6]:
M = 100000 # number of chromosomes
k = 10 # number of ind
mu = 5
t = 0
coal = (1/v_t(t)) * scipy.special.binom(k, 2)

In [9]:
random.seed(10)
T = []
mut = []
nm = 0 # number of mut
nc = 0 # number of coal
w = rate(mu, k, v_t(t))
t = 0
plt.figure(figsize=(5,8)) #
#plt.axis('equal')
while k > 1:
    t += np.random.exponential(1 / w) #time is sampled till the next event from Poisson process with the rate w
    if np.random.binomial(1, k*mu/w): #we use the binomial distribution with n = 1 to have a bernouli distribution
        i = random.randint(0, k-1)
        place = random.random()
        mut.append([i, place, t]) # индивидуум, позиция, время
        mut1 = np.asarray(mut)
        #plt.scatter(mut1[nm][0], mut1[nm][2], color = 'r')
        nm += 1
    else:
        i, j = np.random.choice(k, 2, replace=False) # между кем коалесценция
        if i != j:
            T.append([i, j, t]) # добавляем в список эту инфу
        T1 = np.asarray(T) # превращаем в массив
        nc += 1
    
    k -= 1 # уменьшаем K

print("Number of coal.: ", nc)
print("coalescent between i and j at the moment t:")
print(np.asarray(T))
print('\n')
print("Number of mut.: ", nm)
print("mutation in the individue i in the place p at the moment t:")
print(np.array(mut))

Number of coal.:  0
coalescent between i and j at the moment t:
[[3.         2.         0.0148212 ]
 [5.         7.         0.02928279]
 [2.         1.         0.03413966]
 [4.         3.         0.0435805 ]
 [0.         2.         0.07024701]
 [0.         1.         0.15226556]]


Number of mut.:  0
mutation in the individue i in the place p at the moment t:
[[0.         0.42888905 0.03164877]
 [4.         0.01483245 0.0318236 ]
 [1.         0.81332125 0.11878317]]


<Figure size 360x576 with 0 Axes>

## Useful links

https://en.wikipedia.org/wiki/Inverse_transform_sampling

In [8]:
'''
func name
Input: ...
Output: ...
'''

'\nfunc name\nInput: ...\nOutput: ...\n'